# Before you start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [2]:
# import numpy and pandas
import scipy.stats as st
import numpy as np
import pandas as pd


# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. 

In [5]:

data = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')


Examine the `salaries` dataset using the `head` function below.

In [6]:

data.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"ADRIANO, RACQUEL ANNE",PARAMEDIC,FIRE,F,Salary,NaN,72510.0,NaN
1,"AFFANEH, MAHIR A",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN
2,"AFOENYI, PHYLLIS",TITLE V PROGRAM TRAINEE I,FAMILY & SUPPORT,P,Hourly,20.0,NaN,13.0
3,"AGAR, BULENT B",DEPUTY COMMISSIONER,WATER MGMNT,F,Salary,NaN,132972.0,NaN
4,"AGEE, LAWRENCE",POLICE OFFICER,POLICE,F,Salary,NaN,90024.0,NaN


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [7]:
# Your code here:
data.isna().sum()


Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25528
Annual Salary         8174
Hourly Rate          25528
dtype: int64

In [8]:
data.dtypes

Name                  object
Job Titles            object
Department            object
Full or Part-Time     object
Salary or Hourly      object
Typical Hours        float64
Annual Salary        float64
Hourly Rate          float64
dtype: object

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [9]:
# Your code here:
data['Salary or Hourly'].value_counts()


Salary    25528
Hourly     8174
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [17]:
# Your code here:

department = data.groupby(['Department']).agg({'Name':pd.Series.nunique}).rename(columns={'Name':'count_employees'})
department = department.sort_values(by='count_employees', ascending=False)
department


,count_employees
Department,
POLICE,13784
FIRE,4615
STREETS & SAN,2170
WATER MGMNT,1912
AVIATION,1855
OEMC,1737
TRANSPORTN,1111
PUBLIC LIBRARY,1088
DAIS,1032


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [70]:

# Ho = hourly wage of all hourly workers = $30/hr
# H1 = hourly wage of all hourly workers <> $30/hr
# alpha = 0.05 (p-value threshold)

from scipy.stats import ttest_1samp

data_hourly = data['Hourly Rate'].dropna()


stat, pval = ttest_1samp(data_hourly, 30)

print('stat is  ', stat)
print('pvalue for the two-tailed test is ', pval)


stat is   31.211500329216044
pvalue for the two-tailed test is  3.825475847673794e-202


In [31]:
np.mean(data_hourly)

34.58644971862046

 - Since p-value is less than 0.05, we can reject the null hypotheses.
 - Average hourly rate is 34.58, different from 30 euros/hour 

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [41]:
# Ho = salaries in the police force <= $86000/year
# H1 = salaries in the police force > $86000/year
# alpha = 0.05 (p-value threshold)

data_police_salary = data[data.Department == 'POLICE']
data_police_salary = data_police_salary['Annual Salary'].dropna()

stat, pval = ttest_1samp(data_police_salary, 86000)

print('stat is  ', stat)
print('pvalue for the one-tailed test is ', pval/2)



stat is   18.51748561981861
pvalue for the one-tailed test is  6.119566952121482e-76


In [42]:
np.mean(data_police_salary)

88834.11892361111

 - Since p-value is less than 0.05, we can reject the null hypotheses.
 - Average annual salary in the police force is aprox. 88k euros, greater than 86k 

Using the `crosstab` function, find the department that has the most hourly workers. 

In [58]:

data_crosstab = pd.crosstab(data['Department'], data['Salary or Hourly'], margins = False)

data_crosstab = data_crosstab.pivot_table(index= ['Department'], values = ['Hourly'], aggfunc = {'Hourly':max})
data_crosstab = data_crosstab.sort_values(by='Hourly', ascending=False)
data_crosstab.head(1)

Salary or Hourly,Hourly
Department,
STREETS & SAN,1846


- STREETS & SAN is the department with the highest number of hourly employees

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [68]:
# Ho = hourly wage of STREETS & SAN hourly workers < $35/hr
# H1 = hourly wage of all hourly workers >= $35/hr
# alpha = 0.05 (p-value threshold)

data_STREETS_SAN_rate = data[data.Department == 'STREETS & SAN']
data_STREETS_SAN_rate = data['Hourly Rate'].dropna()
data_STREETS_SAN_rate


stat, pval = ttest_1samp(data_STREETS_SAN_rate, 35)

print('stat is  ', stat)
print('pvalue for the one-tailed test is ', pval/2)




stat is   -2.8142736834194233
pvalue for the one-tailed test is  0.002450222172329931


In [69]:
np.mean(data_STREETS_SAN_rate)

34.58644971862046

- Since p-value is less than 0.05, we can reject the null hypotheses.
- Average hourly rate for STREETS & SAN is aprox. 34.59 euros - hours, less than 86k 

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [73]:

import scipy.stats

confidence_level = 0.95

data_salary = data['Hourly Rate'].dropna()
degrees_freedom = len(data_hourly)

sample_mean = np.mean(data_hourly)

sample_standard_error = scipy.stats.sem(data_hourly)

confidence_interval = scipy.stats.t.interval(confidence_level, 
                                             degrees_freedom, 
                                             sample_mean, 
                                             sample_standard_error)

print( 'confidence interval is ', confidence_interval, '.' )
print('mean hourly wage of all hourly workers: ',sample_mean)

confidence interval is  (34.2983953972632, 34.874504039977715) .
mean hourly wage of all hourly workers:  34.58644971862046


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [74]:
import scipy.stats

confidence_level = 0.95


degrees_freedom = len(data_police_salary)

sample_mean = np.mean(data_police_salary)

sample_standard_error = scipy.stats.sem(data_police_salary)

confidence_interval = scipy.stats.t.interval(confidence_level, 
                                             degrees_freedom, 
                                             sample_mean, 
                                             sample_standard_error)

print( 'confidence interval is ', confidence_interval, '.' )
print('mean hourly wage of all hourly workers: ',sample_mean)

confidence interval is  (88534.11829048868, 89134.11955673354) .
mean hourly wage of all hourly workers:  88834.11892361111


# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

#### the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

Ho: p = p0 = number of hourly workers in the City of Chicago is equal to 25%


Ho: p <> p0 = number of hourly workers in the City of Chicago is different from 25%


#### statsmodels.stats.proportion.proportions_ztest(count, nobs, value=None, alternative='two-sided', prop_var=False)

Parameters:	
 
 - count (integer or array_like) – the number of successes in nobs trials. If this is array_like, then the assumption is that this represents the number of successes for each independent sample


 - nobs (integer or array-like) – the number of trials or observations, with the same length as count.


 - value (float, array_like or None, optional) – This is the value of the null hypothesis equal to the proportion in the case of a one sample test. In the case of a two-sample test, the null hypothesis is that prop[0] - prop[1] = value, where prop is the proportion in the two samples. If not provided value = 0 and the null is prop[0] = prop[1]


 - alternative (string in ['two-sided', 'smaller', 'larger']) – The alternative hypothesis can be either two-sided or one of the one- sided tests, smaller means that the alternative hypothesis is prop < value and larger means prop > value. In the two sample test, smaller means that the alternative hypothesis is p1 < p2 and larger means p1 > p2 where p1 is the proportion of the first sample and p2 of the second one.


 - prop_var (False or float in (0, 1)) – If prop_var is false, then the variance of the proportion estimate is calculated based on the sample proportion. Alternatively, a proportion can be specified to calculate this variance. Common use case is to use the proportion under the Null hypothesis to specify the variance of the proportion estimate.

In [113]:
p = 0.25
count = len(data)*p
nobs = len(data)
value = data['Salary or Hourly'].value_counts()[1] / len(data)


stat, p_value = proportions_ztest(count, nobs, value)

print('z-value: ', stat)
print('p_value: ',p_value)


if p_value > significance:
   print ("Fail to reject the null hypothesis - we have nothing else to say")
else:
   print ("Reject the null hypothesis - suggest the alternative hypothesis is true")


z-value:  3.163803602504186
p_value:  0.0015572183979645873
Reject the null hypothesis - suggest the alternative hypothesis is true


In [ ]:
#To cross check the result above: use z formula

In [108]:
nominator = value-p

In [109]:
import math 

denominator = math.sqrt(p*(1-p)/len(data))

In [111]:
z = nominator/denominator
z

-3.163803602504186